<a href="https://colab.research.google.com/github/MercyMoparthy/lab-chains-in-langchain/blob/main/lab-chains-in-langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# Load environment variables from .env file
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
#!pip install pandas

In [21]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [22]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [23]:
!pip install langchain_community

In [24]:
%pip install langchain_openai
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

Note: you may need to restart the kernel to use updated packages.


In [25]:
# Replace None by your own value and justify
# Set your OpenAI API key directly here for this notebook session
llm = ChatOpenAI(temperature=0.7, api_key=OPENAI_API_KEY)


In [ ]:
prompt = ChatPromptTemplate.from_template( 
    
    'Provide detailed description with features, availability, price of the {product}'

)

In [ ]:

chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\Mercy\AppData\Local\Temp\ipykernel_41464\546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [ ]:
product = "Pillows Insert"
chain.run(product=product)

C:\Users\Mercy\AppData\Local\Temp\ipykernel_41464\4034771705.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product=product)


'Pillow inserts are essential components of decorative pillows, providing the necessary support and shape to create a comfy and visually appealing accent for your living room or bedroom. Here is a detailed description of pillow inserts, including their features, availability, and price:\n\nFeatures:\n1. Material: Pillow inserts are typically made of hypoallergenic polyester fiberfill or down alternative, offering a plush and fluffy feel.\n2. Size: Pillow inserts come in various sizes to fit different pillow covers, ranging from standard (18x18 inches) to king (20x36 inches).\n3. Support: The inserts are designed to provide firm support and maintain the shape of the pillow, preventing it from looking flat or lumpy.\n4. Zipper closure: Some pillow inserts come with a zipper closure, allowing you to adjust the filling or easily replace the insert when needed.\n5. Easy care: Most pillow inserts are machine washable and can be tumble dried for easy maintenance.\n\nAvailability:\nPillow inse

## SimpleSequentialChain

In [29]:
from langchain.chains import SimpleSequentialChain

In [31]:
llm = ChatOpenAI(temperature=0.9, api_key=OPENAI_API_KEY)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
   'Provide detailed description with features, availability, price of the {product}'
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [32]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    'Provide overview of all the reviews {product}'
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [33]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [34]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Pillow inserts are an essential component of any pillow cover, providing shape, support, and comfort. They are typically made from a variety of materials, such as polyester fiberfill, down feathers, foam, or a combination of these materials. Pillow inserts come in various shapes and sizes to fit different types of pillow covers, including square, rectangular, round, and lumbar.

Features of pillow inserts may include:

1. Hypoallergenic materials: Many pillow inserts are made from hypoallergenic materials, making them suitable for individuals with allergies or sensitivities.
2. Different firmness levels: Pillow inserts are available in different firmness levels, ranging from soft to firm, to suit individual preferences.
3. Zippered or sewn closure: Some pillow inserts come with a zippered closure, allowing for easy removal and replacement of the insert, while others have a sewn closure.
4. Machine washable: Some pillow inserts are machine

'Overall, pillow inserts are an essential accessory for creating a comfortable and stylish home decor. With a variety of sizes, shapes, materials, and features available, there are pillow inserts to suit every preference and style. Whether you prefer a soft and plush pillow insert or a firm and supportive one, there is a wide selection to choose from. With proper care and maintenance, a high-quality pillow insert can provide years of comfort and support for your pillows. Consider investing in a durable and well-made pillow insert to enhance the look and feel of your pillows and overall home decor.'

**Repeat the above twice for different products**

## SequentialChain

In [35]:
from langchain.chains import SequentialChain

In [43]:
llm = ChatOpenAI(temperature=0.9, api_key=OPENAI_API_KEY)


first_prompt = ChatPromptTemplate.from_template(
  'Translate the review to Spanish: {Review}'
)

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key='Spanish_Review'
                    )


In [38]:
second_prompt = ChatPromptTemplate.from_template(
    'Translate the Review into German: {Spanish_Review}'
)

chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key='German_Review' #give a name to this output
                    )


In [39]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    'Translate the Review into Hindi Language: {German_Review}'
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key='Hindi_Review'
                      )


In [40]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        'Here is the HIndi Translation: {Hindi_Review}\nHere is the German Trnaslation: {German_Review}\nHere is the English Translation: {Spanish_Review}\n Based on these translations, provide a brief follow-up message'
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='followup_message'
                     )


In [44]:
# overall_chain: input= Review
# and output= Spanish_Review, German_Review, Hindi_Review, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=['Review'],
    output_variables=['Spanish_Review', 'German_Review', 'Hindi_Review', 'followup_message'],
    verbose=True
)

In [45]:
review = df.Review[5]
overall_chain(review)

C:\Users\Mercy\AppData\Local\Temp\ipykernel_41464\1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'Spanish_Review': 'Encuentro el sabor mediocre. La espuma no se mantiene, es extraño. Compro los mismos en el comercio y el sabor es mucho mejor... ¿Lote viejo o falsificación!?',
 'German_Review': 'Ich finde den Geschmack mittelmäßig. Der Schaum hält nicht, das ist seltsam. Ich kaufe dieselben in Geschäften und der Geschmack ist viel besser... Alte Charge oder Fälschung!?',
 'Hindi_Review': 'मुझे यह स्वाद मध्यम है। फोम नहीं बना, यह अजीब है। मैं दुकानों से एक समान खरीदता हूं और स्वाद बहुत अधिक अच्छा है... पुराना बैच या नकल!?',
 'followup_message': "It seems like there may be some issues with the product you purchased. It's important to be cautious of old batches or counterfeit items. It's always best to purchase from reputable stores to ensure quality."}

**Repeat the above twice for different products or reviews**

## Product[1]: Phone 13 Pro Max

In [47]:
llm = ChatOpenAI(temperature=0.7, api_key=OPENAI_API_KEY)
prompt = ChatPromptTemplate.from_template( 
    'Provide detailed description with features, availability, price of the {product}'
)
chain = LLMChain(llm=llm, prompt=prompt)
product = product[1]
chain.run(product=product)

"Phone 13 Pro Max:\n\nThe iPhone 13 Pro Max is the latest flagship smartphone from Apple, released in September 2021. It comes with a large 6.7-inch Super Retina XDR display with ProMotion technology, which offers a smooth and responsive viewing experience. The phone is powered by the A15 Bionic chip, which is Apple's most powerful processor to date, providing fast and efficient performance for all your daily tasks.\n\nOne of the standout features of the iPhone 13 Pro Max is its camera system, which includes a triple-camera setup with 12MP ultra-wide, wide, and telephoto lenses. This allows for stunning photos and videos, with improved low-light performance and advanced computational photography features. The phone also supports Dolby Vision HDR recording and ProRAW format for even more flexibility in post-processing.\n\nThe iPhone 13 Pro Max also comes with 5G support for faster download and streaming speeds, as well as improved battery life compared to its predecessors. It runs on iO

In [48]:
llm = ChatOpenAI(temperature=0.9, api_key=OPENAI_API_KEY)
first_prompt = ChatPromptTemplate.from_template(
   'Provide detailed description with features, availability, price of the {product}'
)
chain_one = LLMChain(llm=llm, prompt=first_prompt)
second_prompt = ChatPromptTemplate.from_template(
    'Provide overview of all the reviews {product}'
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Phone 13 Pro Max.

The iPhone 13 Pro Max is the latest flagship smartphone from Apple, boasting a plethora of advanced features and impressive specifications. This device features a stunning 6.7-inch Super Retina XDR display with ProMotion technology, offering a smooth and responsive viewing experience with a refresh rate of up to 120Hz.

The iPhone 13 Pro Max is powered by Apple's latest A15 Bionic chip, delivering lightning-fast performance and advanced AI capabilities. This device is available in storage options ranging from 128GB to 1TB, ensuring that you have plenty of space to store your photos, videos, and apps.

In terms of photography, the iPhone 13 Pro Max is equipped with a Triple 12MP camera system, including Wide, Ultra-Wide, and Telephoto lenses. This setup allows you to capture stunning photos and videos in any lighting condition, thanks to features like Night mode, Deep Fusion, and Smart HDR 4.

Other notable features of t

"Overall, reviews of the iPhone 13 Pro Max have been overwhelmingly positive. Critics and users alike have praised its stunning display, powerful performance, and impressive camera capabilities. The device's long battery life, 5G connectivity, and extensive storage options have also been highlighted as key selling points.\n\nSome users have noted that the device's large size may be difficult to handle for those with smaller hands, and the high price point may be a barrier for some potential buyers. However, the overall consensus is that the iPhone 13 Pro Max is a top-of-the-line smartphone that offers a premium experience for those willing to invest in it."

In [51]:
llm = ChatOpenAI(temperature=0.9, api_key=OPENAI_API_KEY)
first_prompt = ChatPromptTemplate.from_template(
  'Translate the review to English: {Review}'
)
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key='English_Review'
                    )
Second_prompt = ChatPromptTemplate.from_template(
  'Translate the review to German: {English_Review}'
)
chain_two = LLMChain(llm=llm, prompt=first_prompt,
                     output_key='German_Review'
                    )
Third_prompt = ChatPromptTemplate.from_template(
  'Translate the review to French: {German_Review}'
)
chain_three = LLMChain(llm=llm, prompt=first_prompt,
                     output_key='French_Review'
                    )
fourth_prompt = ChatPromptTemplate.from_template(
        'Here is the French Translation: {French_Review}\nHere is the German Trnaslation: {German_Review}\nHere is the English Translation: {English_Review}\n Based on these translations, provide a brief follow-up message'
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='followup_message'
                     )
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=['Review'],
    output_variables=['English_Review', 'German_Review', 'French_Review', 'followup_message'],
    verbose=True
)
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste to be mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'German_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'French_Review': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'followup_message': "I recommend reaching out to the seller or manufacturer to inquire about the quality of the product or to express your concerns about a possible issue with the batch. It's important to ensure that you are consuming safe and authentic products."}

## Product [2]: Luxury Air Mattresses

In [70]:
llm = ChatOpenAI(temperature=0.7, api_key=OPENAI_API_KEY)
prompt = ChatPromptTemplate.from_template( 
    'Provide detailed description with features, availability, price of the {product}'
)
chain = LLMChain(llm=llm, prompt=prompt)
product = df.Product[2]
chain.run(product=product)

"Luxury air mattresses are designed to provide the ultimate comfort and support for a restful night's sleep. These mattresses are typically made with high-quality materials and advanced technology to ensure a luxurious sleeping experience. \n\nFeatures of luxury air mattresses may include:\n\n1. High-quality materials: Luxury air mattresses are often made with durable materials such as PVC or TPU that are puncture-resistant and long-lasting.\n\n2. Adjustable firmness: Many luxury air mattresses come with adjustable firmness settings, allowing you to customize the level of support to your liking.\n\n3. Built-in pumps: Some luxury air mattresses come with built-in pumps for easy inflation and deflation, saving you time and effort.\n\n4. Comfort layers: Luxury air mattresses may feature additional comfort layers such as memory foam or gel-infused foam to provide added cushioning and pressure relief.\n\n5. Temperature regulation: Some luxury air mattresses are designed with features to hel

In [71]:
llm = ChatOpenAI(temperature=0.9, api_key=OPENAI_API_KEY)
first_prompt = ChatPromptTemplate.from_template(
   'Provide detailed description with features, availability, price of the {product}'
)
chain_one = LLMChain(llm=llm, prompt=first_prompt)
second_prompt = ChatPromptTemplate.from_template(
    'Provide overview of all the reviews {product}'
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
A luxury air mattress is a high-quality, premium inflatable bed designed for ultimate comfort and support. These mattresses are perfect for camping trips, house guests, or as a primary bed in your home. Here are some key features of a luxury air mattress:

1. Construction: Luxury air mattresses are typically made with durable materials such as puncture-resistant PVC or vinyl. They often have multiple layers for added support and stability.

2. Comfort: These mattresses are designed with comfort in mind, often featuring built-in pillows, plush flocking on the surface, and adjustable firmness settings. Some models even have memory foam or gel-infused layers for extra cushioning.

3. Size options: Luxury air mattresses come in a variety of sizes, including twin, full, queen, and king size. This allows you to choose the size that best fits your needs.

4. Pump: Many luxury air mattresses come with a built-in pump for easy inflation and deflat

'\n\nReviews:\n\n1. "I absolutely love my luxury air mattress! It is so comfortable and easy to inflate. I have used it for camping trips and when guests come over, and everyone always raves about how great it feels to sleep on." - Sarah M.\n\n2. "I was skeptical about purchasing a luxury air mattress at first, but I am so glad I did. The built-in pump is a game-changer and the comfort level is amazing. I feel like I am sleeping on a cloud every time I use it." - John D.\n\n3. "I bought a luxury air mattress for my guest room and it has been a hit with everyone who stays over. The adjustable firmness settings are a nice feature and the size options make it easy to accommodate different guests." - Karen L.\n\n4. "I have used my luxury air mattress on multiple camping trips and it has held up well. The materials are durable and the comfort level is top-notch. I highly recommend investing in a luxury air mattress for all your outdoor adventures." - Mike S.\n\n5. "I have had my luxury air 

In [73]:
llm = ChatOpenAI(temperature=0.9, api_key=OPENAI_API_KEY)
first_prompt = ChatPromptTemplate.from_template(
    "Translate the review to English: {Review}"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key='English_Review')
second_prompt = ChatPromptTemplate.from_template(
    "Translate the review to German: {English_Review}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key='German_Review')
third_prompt = ChatPromptTemplate.from_template(
    "Translate the review to French: {German_Review}"
)
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key='French_Review')
fourth_prompt = ChatPromptTemplate.from_template(
    "Here is the French Translation: {French_Review}\nHere is the German Translation: {German_Review}\nHere is the English Translation: {English_Review}\nBased on these translations, provide a brief follow-up message."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key='followup_message')
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=['Review'],
    output_variables=['English_Review', 'German_Review', 'French_Review', 'followup_message'],
    verbose=True
)

review = df.Review[5]
result = overall_chain({'Review': review})
print(result)




> Entering new SequentialChain chain...

> Finished chain.
{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?", 'English_Review': "I find the taste mediocre. The foam doesn't last, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?", 'German_Review': 'Ich finde den Geschmack mittelmäßig. Der Schaum hält nicht lange, das ist seltsam. Ich kaufe die gleichen in Geschäften und der Geschmack ist viel besser... Alte Charge oder Fälschung!?', 'French_Review': "Je trouve le goût moyen. La mousse ne tient pas longtemps, c'est étrange. J'achète les mêmes en magasin et le goût est bien meilleur... Ancien lot ou contrefaçon?", 'followup_message': "It seems like there may be a quality issue with the product you purchased. It's important to be cautious when buying items that may be counterfeit or from an old batch. It might

## Router Chain

In [53]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [54]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [55]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [56]:
llm = ChatOpenAI(temperature=0)

In [57]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [58]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [59]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [60]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [61]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

C:\Users\Mercy\AppData\Local\Temp\ipykernel_41464\2333002322.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [62]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [63]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'Thank you for the compliment! The answer to the question "what is 2 + 2" is 4.'

In [64]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, DNA is constantly being used by cells to carry out processes such as protein synthesis, cell division, and repair. Having DNA in every cell allows for the coordination of these processes and ensures the proper functioning of the organism as a whole. \n\nIn summary, every cell in our body contains DNA because it is essential for the development, functioning, and reproduction of all living organisms.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [65]:
chain.run("What is the largest city in the world")



> Entering new MultiPromptChain chain...
None: {'input': 'What is the largest city in the world'}
> Finished chain.


'The largest city in the world by population is Tokyo, Japan.'

In [66]:
chain.run("What is 6789 + 2234?")



> Entering new MultiPromptChain chain...
math: {'input': 'What is 6789 + 2234?'}
> Finished chain.


'To solve this problem, we can break it down into two separate addition problems:\n\n1. 6000 + 2000 = 8000\n2. 789 + 234 = 1023\n\nNow, we can add these two results together:\n\n8000 + 1023 = 9023\n\nTherefore, 6789 + 2234 is equal to 9023.'

In [67]:
chain.run("What is the difference between a stack and a queue in data structures?")



> Entering new MultiPromptChain chain...
computer science: {'input': 'What is the difference between a stack and a queue in data structures?'}
> Finished chain.


'A stack and a queue are both data structures that store elements in a specific order, but they differ in how elements are added and removed.\n\n1. Stack:\n- A stack is a Last In, First Out (LIFO) data structure, meaning that the last element added to the stack is the first one to be removed.\n- Elements are added and removed from the top of the stack.\n- Common operations on a stack include push (add an element to the top of the stack) and pop (remove the top element from the stack).\n\n2. Queue:\n- A queue is a First In, First Out (FIFO) data structure, meaning that the first element added to the queue is the first one to be removed.\n- Elements are added to the back of the queue and removed from the front.\n- Common operations on a queue include enqueue (add an element to the back of the queue) and dequeue (remove the front element from the queue).\n\nIn summary, the main difference between a stack and a queue is the order in which elements are added and removed. A stack follows the